In [8]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.metrics import *
import torch
from torch import nn
from torch.utils.data import *
from torchvision import datasets
from torchvision.transforms import *


In [9]:


root_results_dir = "/Users/newuser/Projects/robust-algo-trader/data/trades_seq_fixed_EURUSD_H1_2011_2023.csv"
df = pd.read_csv(f"{root_results_dir}")

y = df["label"]
X = df[["position", "RSI",  "ADX", "AROON_Oscillator"]]



In [10]:
df

,index,ask_price,take_profit_price,stop_loss_price,position,SMA_200,MACD,MACD_Signal,MACD_Hist,MACD_Crossover_Change,...,WILLR,OBV,CCI,PSAR,AD,ADOSC,VOLUME_RSI,MFI,Date_Time,label
0,209,1.30870,1.31270,1.30620,1,1.316625,-0.001955,-0.001962,0.000007,2.0,...,-79.268293,2987.0,-59.563728,1.310000,-3.518072e+03,-314.606376,43.118997,27.493133,2007.03.05 22:00:00,1
1,244,1.31130,1.30730,1.31380,0,1.316453,0.000262,0.000342,-0.000080,-2.0,...,-81.818182,4903.0,-138.344595,1.313834,-3.619358e+03,-94.962257,52.352590,46.481357,2007.03.07 09:00:00,0
2,265,1.31710,1.31310,1.31960,0,1.316768,0.001249,0.001289,-0.000040,-2.0,...,-28.571429,6683.0,29.626780,1.318460,-2.647393e+03,9.642566,50.225690,45.481383,2007.03.08 06:00:00,1
3,285,1.31430,1.31830,1.31180,1,1.316485,-0.000365,-0.000393,0.000028,2.0,...,-50.909091,6143.0,74.769797,1.311746,-2.825622e+03,134.357644,47.591747,44.034916,2007.03.09 02:00:00,0
4,307,1.31210,1.31610,1.30960,1,1.315822,-0.000718,-0.000724,0.000006,2.0,...,-52.054795,4983.0,3.345281,1.313866,-2.560048e+03,135.647302,48.192723,45.661289,2007.03.12 01:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4134,100000,1.10489,1.10889,1.10239,1,1.091827,0.001867,0.001686,0.000181,2.0,...,-6.225681,15049.0,177.792861,1.098382,-3.904942e+06,1065.826390,66.431461,62.992627,2023.04.13 15:00:00,0
4135,100042,1.09868,1.10268,1.09618,1,1.094480,-0.001327,-0.001374,0.000047,2.0,...,-33.244681,-25520.0,24.394401,1.099470,-3.909506e+06,-281.560849,52.198844,48.281841,2023.04.17 09:00:00,0
4136,100058,1.09241,1.09641,1.08991,1,1.094418,-0.002050,-0.002051,0.000001,2.0,...,-80.449141,-43695.0,-42.238600,1.096220,-3.925679e+06,-1665.883694,40.977494,28.144375,2023.04.18 01:00:00,1
4137,100088,1.09686,1.09286,1.09936,0,1.095222,0.000451,0.000463,-0.000012,-2.0,...,-52.559727,-20871.0,-17.138055,1.098362,-3.917172e+06,781.721024,42.811481,56.587638,2023.04.19 07:00:00,0


In [11]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))
y_test = torch.tensor(y_test.values.astype(np.float32))

batch_size = 128

# Create DataLoader objects for training and testing
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [12]:
y_train

tensor([1., 0., 1.,  ..., 0., 1., 0.])

In [13]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    
input_size = len(X.columns)
hidden_size = 64
output_size = 1
model = NeuralNetwork().to(device)
# loss_fn = PrecisionLoss()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=1, bias=True)
    (9): Sigmoid()
  )
)


In [ ]:


# Define test function
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            label = y.float().view(-1, 1)
            test_loss += criterion(pred,label).item()
            correct += (pred > 0.5).eq(y.view_as(pred)).sum().item()
    test_loss /= num_batches
    # correct /= size
    print(f"Test Loss: {test_loss}")

# Train and test the model
epochs = 500_000_000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train()
    loss_sum = 0
    correct = 0
    for batch, (X, y) in enumerate(train_dataloader):
        # Compute prediction error
        X, y = X.to(device), y.to(device)
        pred = model(X)
        # print(pred)
        label = y.float().view(-1, 1)
        loss = loss_fn(pred, label)

        optimizer.zero_grad()
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        loss_sum += loss.item()
        
        # if batch % 100 == 0:
            # loss, current = loss.item(), (batch + 1) * len(X)
      
    print(f"Train Loss: {loss_sum / len(train_dataloader)}")
    
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list to store train and test accuracies for each epoch
train_accuracies = []
test_accuracies = []

# Your training loop with accuracy calculations during each epoch
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_acc = test(test_dataloader, model, loss_fn)
    
    # Append accuracies to the lists
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

# Plot the graph
epochs_range = range(1, epochs+1)
plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
plt.plot(epochs_range, test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy Over Epochs')
plt.legend()
plt.show()



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))


# test_dataloader
# calculate the accuracy over the whole test set
